In [109]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [110]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df_normal_marked = pd.read_csv('saved.csv')
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df.index = df.index + 1

In [3]:
df_final = pd.concat([df, df_normal_marked], axis=0)

In [4]:
df_final = df_final.reset_index(drop=True)
del df_final["Unnamed: 0"]

In [111]:
df_final
for index, row in df_final.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
    im.save("png_converted/{}.png".format(row['ID']))

AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [6]:
df_normal = pd.read_csv('data.txt', sep=" ", header=None)
df_normal.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]

In [7]:
df_normal = df_normal.loc[df_normal['class']  == 'NORM']
df_normal = df_normal.reset_index(drop=True)

In [8]:
df_normal

,ID,background_tissue,class,severity,x,y,rad
0,mdb003,D,NORM,NaN,NaN,NaN,NaN
1,mdb004,D,NORM,NaN,NaN,NaN,NaN
2,mdb006,F,NORM,NaN,NaN,NaN,NaN
3,mdb007,G,NORM,NaN,NaN,NaN,NaN
4,mdb008,G,NORM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
202,mdb318,D,NORM,NaN,NaN,NaN,NaN
203,mdb319,D,NORM,NaN,NaN,NaN,NaN
204,mdb320,D,NORM,NaN,NaN,NaN,NaN
205,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [9]:
df['severity'].value_counts()

B    68
M    51
Name: severity, dtype: int64

In [10]:
im = Image.open('all-mias/mdb001.pgm') # PILLOW image processing

In [11]:
im.getpixel( (535,425) )

155

In [110]:
last = 'start'

In [111]:
def img_draw(image, x, y, r, c):
    if c=='M':
        color = 'red'
    else:
        color = 'blue'
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, outline=color, width=5)

In [ ]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [19]:
mean = []
var = []
std = []
max_v = []
min_v = []
min_coor_x = []
min_coor_y = []
max_coor_x = []
max_coor_y = []
for index, row in df_final.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    #c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    
    mean.append(cv2.meanStdDev(im, mask = mask)[0][0])
    var.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    std.append(cv2.meanStdDev(im, mask = mask)[1][0])
    
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray, mask = mask)
    
    max_v.append(max_val)
    min_v.append(min_val)
    min_coor_x.append(min_loc[0])
    min_coor_y.append(min_loc[1])
    max_coor_x.append(max_loc[0])
    max_coor_y.append(max_loc[1])


In [20]:
df_final['mean'] = np.array(mean)
df_final['var'] = np.array(var)
df_final['std'] = np.array(std)
df_final['min_pixel'] = np.array(min_v)
df_final['max_pixel'] = np.array(max_v)
df_final['min_coor_x'] = np.array(min_coor_x)
df_final['min_coor_y'] = np.array(min_coor_y)
df_final['max_coor_x'] = np.array(max_coor_x)
df_final['max_coor_y'] = np.array(max_coor_y)

In [98]:
df_final.to_csv("final.csv")

In [21]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,min_pixel,max_pixel,min_coor_x,min_coor_y,max_coor_x,max_coor_y
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,5.0,221.0,373,710,487,632
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,75.0,229.0,571,792,531,679
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,84.0,187.0,481,920,476,876
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,128.0,181.0,520,841,522,859
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,155.0,208.0,557,599,519,598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,NaN,442,68,50.0,14.033015,0.505857,0.711236,11.0,35.0,436,951,467,965
308,mdb319,D,NORM,NaN,590,281,50.0,145.912938,28.130725,5.303841,130.0,168.0,587,778,637,745
309,mdb320,D,NORM,NaN,495,264,50.0,195.578840,470.810515,21.698168,146.0,227.0,451,745,527,722
310,mdb321,D,NORM,NaN,604,271,50.0,177.764946,30.464061,5.519426,163.0,199.0,626,740,555,746


In [76]:
all_coor = []
def get_all_coor():
    for index, row in df_final.iterrows():
        coordinates = []
        for x in range( int(row['x']) - int(row['rad']) ,  int(row['x']) + int(row['rad']) ) :
            for y in range( int(row['y']) - int(row['rad']), int(row['y']) + int(row['rad']) ):
                dx = x - int(row['x'])
                dy = y - int(row['y'])
                distanceSquared = dx * dx + dy * dy
                if (distanceSquared <= row['rad']**2):
                    coordinates.append((x,y))
        all_coor.append(coordinates)

In [77]:
get_all_coor()

In [101]:
bdips = []
def bdip():    
    for index, row in df_final.iterrows():
        path = 'all-mias/{name}.pgm'.format(name=row['ID'])
        im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
        s = 0
        for x, y in all_coor[index]:
            s += im[row['max_coor_y'],row['max_coor_x']] - im[y, x]
        bdips.append( (s[0]/(row['rad']**2*3.14*im[row['max_coor_y'],row['max_coor_x']]))[0] )    

In [80]:
im = cv2.imread('all-mias/mdb001.pgm')

In [92]:
s = 0
for x, y in all_coor[0]:
    s += im[632,487] - im[y, x]

In [94]:
(1/(221*(197)**2*3.14))*s[0]

7.017904113363145e-06

In [102]:
bdip()
bdips

<ipython-input-101-e75bd423e2a1>:9: RuntimeWarning: divide by zero encountered in true_divide
  bdips.append( (s[0]/(row['rad']**2*3.14*im[row['max_coor_y'],row['max_coor_x']]))[0] )


[7.017904113363144e-06,
 5.5207552266961426e-05,
 8.704504770447071e-05,
 0.0005778272985605542,
 0.00028681991623508705,
 4.312632696390658e-05,
 0.00013992265959099872,
 2.0431390757123542e-05,
 7.64380130519436e-05,
 3.908748084505186e-05,
 5.823268837611484e-06,
 0.00044495103645190363,
 5.72217227610802e-05,
 6.475191315350425e-05,
 0.00014695686929659202,
 5.972904243025841e-05,
 0.00048161211343968437,
 0.00014687179327210208,
 0.00012753344259560796,
 0.00030328152819187443,
 0.0003010126064079563,
 3.184713375796178e-05,
 1.0798751422443534e-05,
 9.173236135024416e-05,
 7.596187690765837e-06,
 8.893481501558476e-05,
 0.00011694173962531865,
 0.0002559608011110241,
 0.00024503758738151114,
 7.326232748553434e-05,
 9.255140029087133e-05,
 9.216589861751152e-05,
 6.166167932388193e-06,
 1.2641742347416648e-05,
 0.00012244188296025292,
 1.4337149079374529e-05,
 6.197163882375416e-06,
 2.1401960104237464e-05,
 2.2733252078241257e-05,
 1.303231136702996e-05,
 0.0003206490792428086,


In [104]:
df_final['bdips'] = np.array(bdips)

In [84]:
df_final.head(1)

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,min_pixel,max_pixel,min_coor_x,min_coor_y,max_coor_x,max_coor_y
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,5.0,221.0,373,710,487,632


312

In [105]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,min_pixel,max_pixel,min_coor_x,min_coor_y,max_coor_x,max_coor_y,bdips
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,5.0,221.0,373,710,487,632,0.000007
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,75.0,229.0,571,792,531,679,0.000055
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,84.0,187.0,481,920,476,876,0.000087
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,128.0,181.0,520,841,522,859,0.000578
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,155.0,208.0,557,599,519,598,0.000287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,NaN,442,68,50.0,14.033015,0.505857,0.711236,11.0,35.0,436,951,467,965,0.000641
308,mdb319,D,NORM,NaN,590,281,50.0,145.912938,28.130725,5.303841,130.0,168.0,587,778,637,745,0.000114
309,mdb320,D,NORM,NaN,495,264,50.0,195.578840,470.810515,21.698168,146.0,227.0,451,745,527,722,0.000004
310,mdb321,D,NORM,NaN,604,271,50.0,177.764946,30.464061,5.519426,163.0,199.0,626,740,555,746,0.000026


In [106]:
df_final.to_csv("final.csv")